# Tratamiento de los datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import NearestNeighbors

import geopandas as gpd
from shapely.geometry import Point
from scipy.optimize import minimize

In [ ]:
# Estilo gráfico
sns.set(style="whitegrid")

In [ ]:
# Importar datos

dim_tienda = pd.read_csv("/Users/abigail/Desktop/SEM VIII/DSC/Data/DIM_TIENDA.csv")
venta = pd.read_csv("/Users/abigail/Desktop/SEM VIII/DSC/Data/Venta.csv")
meta_venta = pd.read_csv("/Users/abigail/Desktop/SEM VIII/DSC/Data/Meta_venta.csv")

In [ ]:
# Merge de datasets
df = venta.merge(dim_tienda, on="TIENDA_ID", how="left")
df = df.merge(meta_venta, on="ENTORNO_DES", how="left")


In [ ]:
# Crear variable target
df["EXITO"] = (df["VENTA_TOTAL"] >= df["Meta_venta"]).astype(int)

In [ ]:
# Mostrar estructura del dataframe final
df

# EDA

## Unicas Tiendas Agrupacion

In [ ]:
# Agrupar por tienda única y SUMA 
df_tiendas = df.groupby("TIENDA_ID", as_index=False).agg({
    "VENTA_TOTAL": "sum",
    "Meta_venta": "mean",
    "MTS2VENTAS_NUM": "mean",
    "PUERTASREFRIG_NUM": "mean",
    "CAJONESESTACIONAMIENTO_NUM": "mean",
    "LATITUD_NUM": "first",
    "LONGITUD_NUM": "first",
    "EXITO": "max",  
    "PLAZA_CVE": "first",
    "NIVELSOCIOECONOMICO_DES": "first",
    "ENTORNO_DES": "first",
    "SEGMENTO_MAESTRO_DESC": "first",
    "LID_UBICACION_TIENDA": "first"
})


In [ ]:
df_tiendas

## Tiendas con 0mt2

In [ ]:
# Filtramos las tiendas con 0 m2
tiendas_mt2_cero = df[df["MTS2VENTAS_NUM"] == 0]

# Extraemos IDs únicos
tiendas_unicas_cero = tiendas_mt2_cero["TIENDA_ID"].nunique()

print(f"Número de tiendas únicas con 0 m2: {tiendas_unicas_cero}")


In [ ]:
# Mostrar los IDs únicos
tiendas_unicas_lista = tiendas_mt2_cero["TIENDA_ID"].unique()
tiendas_unicas_lista.sort()
print("Tiendas con 0 m²:")
print(tiendas_unicas_lista)


## Estadisticas

In [ ]:
# CORRELACION CON EXITO
correlaciones = df_tiendas.corr(numeric_only=True)
print(correlaciones["EXITO"].sort_values(ascending=False))


In [ ]:
# Nivel Socioeconomico y exitos
plt.figure(figsize=(10,5))
sns.countplot(data=df_tiendas, x="NIVELSOCIOECONOMICO_DES", hue="EXITO", palette="Set2")
plt.title("Éxito por Nivel Socioeconómico")
plt.xticks(rotation=45)
plt.xlabel("Nivel Socioeconómico")
plt.ylabel("Número de Tiendas")
ax = sns.countplot(data=df_tiendas, x="NIVELSOCIOECONOMICO_DES", hue="EXITO", palette="Set2")
for bar in ax.patches:
    height = bar.get_height()
    width = bar.get_width()
    x = bar.get_x()
    ax.text(x + width / 2, height + 1, f"{int(height)}", ha='center', va='bottom', fontsize=10)
plt.legend(title="¿Cumple Meta?", labels=["No", "Sí"])
plt.show()

# La suma total de ventas por tienda


In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(data=df_tiendas, x="ENTORNO_DES", hue="EXITO", palette="Set2")
plt.title("Conteo de tiendas por ENTORNO_DES y Éxito")
plt.xticks(rotation=45)
plt.legend(title="Éxito", labels=["No", "Sí"])

for bar in ax.patches:
    height = bar.get_height()
    width = bar.get_width()
    x = bar.get_x()
    ax.text(x + width / 2, height + 1, f"{int(height)}", ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(data=df_tiendas, x="SEGMENTO_MAESTRO_DESC", hue="EXITO", palette="Set2")
plt.title("Conteo de tiendas por SEGMENTO_MAESTRO_DESC y Éxito")
plt.xticks(rotation=45)
plt.legend(title="Éxito", labels=["No", "Sí"])

for bar in ax.patches:
    height = bar.get_height()
    width = bar.get_width()
    x = bar.get_x()
    ax.text(x + width / 2, height + 1, f"{int(height)}", ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(data=df_tiendas, x="LID_UBICACION_TIENDA", hue="EXITO", palette="Set2")
plt.title("Conteo de tiendas por LID_UBICACION_TIENDA y Éxito")
plt.xticks(rotation=45)
plt.legend(title="Éxito", labels=["No", "Sí"])

for bar in ax.patches:
    height = bar.get_height()
    width = bar.get_width()
    x = bar.get_x()
    ax.text(x + width / 2, height + 1, f"{int(height)}", ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
sns.kdeplot(data=df_tiendas, x="MTS2VENTAS_NUM", hue="EXITO", common_norm=False)


In [ ]:
sns.kdeplot(data=df_tiendas, x="CAJONESESTACIONAMIENTO_NUM", hue="EXITO", common_norm=False)

In [ ]:
sns.kdeplot(data=df_tiendas, x="PUERTASREFRIG_NUM", hue="EXITO", common_norm=False)

### Proporcion sankey

In [ ]:
# Data
df_sankey = df_tiendas[[
    "NIVELSOCIOECONOMICO_DES",
    "ENTORNO_DES",
    "SEGMENTO_MAESTRO_DESC",
    "LID_UBICACION_TIENDA",
    "EXITO"
]].copy()

In [ ]:
# Mapear EXITO a texto
df_sankey["EXITO"] = df_sankey["EXITO"].map({0: "No Exitoso", 1: "Exitoso"})

In [ ]:
# Lista única de etiquetas
labels = pd.concat([
    df_sankey["NIVELSOCIOECONOMICO_DES"],
    df_sankey["ENTORNO_DES"],
    df_sankey["SEGMENTO_MAESTRO_DESC"],
    df_sankey["LID_UBICACION_TIENDA"],
    df_sankey["EXITO"]
]).unique().tolist()

In [ ]:
# Función para obtener índice
def get_index(label):
    return labels.index(label)

In [ ]:
# Total para porcentaje
total = len(df_sankey)

In [ ]:
# Flujos entre columnas
def make_links(source_col, target_col):
    group = df_sankey.groupby([source_col, target_col]).size().reset_index(name='count')
    group["percentage"] = group["count"] / total * 100
    group["label"] = group.apply(lambda row: f"{row['count']} tiendas ({row['percentage']:.1f}%)", axis=1)
    sources = group[source_col].apply(get_index).tolist()
    targets = group[target_col].apply(get_index).tolist()
    values = group["count"].tolist()
    labels_hover = group["label"].tolist()
    return sources, targets, values, labels_hover

In [ ]:
# Generar los datos para cada nivel
s1, t1, v1, l1 = make_links("NIVELSOCIOECONOMICO_DES", "ENTORNO_DES")
s2, t2, v2, l2 = make_links("ENTORNO_DES", "SEGMENTO_MAESTRO_DESC")
s3, t3, v3, l3 = make_links("SEGMENTO_MAESTRO_DESC", "LID_UBICACION_TIENDA")
s4, t4, v4, l4 = make_links("LID_UBICACION_TIENDA", "EXITO")

In [ ]:
# Unir todo
sources = s1 + s2 + s3 + s4
targets = t1 + t2 + t3 + t4
values = v1 + v2 + v3 + v4
hover_labels = l1 + l2 + l3 + l4

In [ ]:
# Crear Sankey
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        label=labels,
        color="lightblue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        label=hover_labels,
        customdata=hover_labels,
        hovertemplate='<b>%{customdata}</b><extra></extra>'
    ))])

In [ ]:

fig.update_layout(title_text="Sankey Multinivel con Conteo y Porcentaje de Tiendas", font_size=10)
fig.show()


# Limpieza de datos

In [ ]:
# Mt2 = 0
media_mt2 = df.loc[df["MTS2VENTAS_NUM"] > 0, "MTS2VENTAS_NUM"].mean()
df.loc[df["MTS2VENTAS_NUM"] == 0, "MTS2VENTAS_NUM"] = media_mt2

In [ ]:
# Refris = 0
df["PUERTASREFRIG_NUM"] = df.groupby(
    ["ENTORNO_DES", "SEGMENTO_MAESTRO_DESC"]
)["PUERTASREFRIG_NUM"].transform(
    lambda x: x.replace(0, x[x > 0].mean())
)

In [ ]:
# NULL BUSTING 
df_clean = df.dropna()

In [ ]:
df_clean.isnull().sum()

In [ ]:
df_clean

# Exploracion

## Mapa

In [ ]:
# Inicializar el mapa y el cluster
mapa = folium.Map(location=[df_clean["LATITUD_NUM"].mean(), df_clean["LONGITUD_NUM"].mean()], zoom_start=6)
marker_cluster = MarkerCluster().add_to(mapa)

In [ ]:
# Coordenadas
coordenadas_agregadas = set()

In [ ]:
# Mapear
for _, row in df_clean.iterrows():
    coord = (row["LATITUD_NUM"], row["LONGITUD_NUM"])
    
    if coord in coordenadas_agregadas:
        continue  

    coordenadas_agregadas.add(coord)
    
    color = "green" if row["EXITO"] == 1 else "red"
    folium.CircleMarker(
        location=[row["LATITUD_NUM"], row["LONGITUD_NUM"]],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Tienda: {row['TIENDA_ID']}",
    ).add_to(marker_cluster)

In [ ]:
# Mostrar mapa
mapa


# Modelo?

## Base

In [ ]:
df_tiendas = df.groupby('TIENDA_ID', as_index=False).agg({
    'VENTA_TOTAL': 'mean',
    'Meta_venta': 'mean',
    'MTS2VENTAS_NUM': 'mean',
    'PUERTASREFRIG_NUM': 'mean',
    'CAJONESESTACIONAMIENTO_NUM': 'mean',
    'LATITUD_NUM': 'first',
    'LONGITUD_NUM': 'first',
    'PLAZA_CVE': 'first',
    'NIVELSOCIOECONOMICO_DES': 'first',
    'ENTORNO_DES': 'first',
    'SEGMENTO_MAESTRO_DESC': 'first',
    'LID_UBICACION_TIENDA': 'first'
})

df_tiendas['EXITO'] = (df_tiendas['VENTA_TOTAL'] >= df_tiendas['Meta_venta']).astype(int)
df_tiendas['PCT_CUMPLIMIENTO'] = df_tiendas['VENTA_TOTAL'] / df_tiendas['Meta_venta'] * 100




In [ ]:
df_tiendas

In [ ]:
# Mt2 = 0
media_mt2 = df_tiendas.loc[df_tiendas["MTS2VENTAS_NUM"] > 0, "MTS2VENTAS_NUM"].mean()
df_tiendas.loc[df_tiendas["MTS2VENTAS_NUM"] == 0, "MTS2VENTAS_NUM"] = media_mt2

In [ ]:
# Refris = 0
df_tiendas["PUERTASREFRIG_NUM"] = df_tiendas.groupby(
    ["ENTORNO_DES", "SEGMENTO_MAESTRO_DESC"]
)["PUERTASREFRIG_NUM"].transform(
    lambda x: x.replace(0, x[x > 0].mean())
)

In [ ]:
# NULL BUSTING 
df_clean = df_tiendas.dropna()

In [ ]:
df_clean.isnull().sum()

In [ ]:
df_clean

## Prediccion normalita

In [ ]:
# Funcion de distancia Haversine

def haversine(lat1, lon1, lat2, lon2):
    # Radio de la Tierra en kilómetros
    R = 6371  

    # Convierto tooooooodas las coordenadas de grados a radianes (porque las funciones trigonométricas usan radianes)
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Calculo la diferencia entre las latitudes y longitudes
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine: calculo la distancia entre dos puntos en una esfera (asumiendo una Tierra esférica, right?)
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # Multiplico por el radio de la Tierra para convertir la distancia angular a kilómetros
    return R * c



In [ ]:
# Ruta 
ruta_shapefile = "/Users/abigail/Desktop/SEM VIII/DSC/Data/ne_10m_admin_1_states_provinces/ne_10m_admin_1_states_provinces.shp"

# Cargar el shapefile
gdf = gpd.read_file(ruta_shapefile)

# Filtrar sólo México
mexico = gdf[gdf['admin'] == 'Mexico']

def esta_en_mexico(lat, lon):
    # shapely usa (lon, lat)
    punto = Point(lon, lat)  
    return mexico.contains(punto).any()


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
import pandas as pd

def entrenar_modelo(df):

    # Columnas categóricas y numéricas
    cat_cols = ["PLAZA_CVE", "NIVELSOCIOECONOMICO_DES", "ENTORNO_DES", "SEGMENTO_MAESTRO_DESC", "LID_UBICACION_TIENDA"]
    num_cols = ["MTS2VENTAS_NUM", "PUERTASREFRIG_NUM", "CAJONESESTACIONAMIENTO_NUM", "LATITUD_NUM", "LONGITUD_NUM"]

    # Limpieza
    df = df.dropna(subset=cat_cols + num_cols)
    X_cat = df[cat_cols]
    X_num = df[num_cols]

    # Codificamos categorías
    encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
    X_cat_encoded = encoder.fit_transform(X_cat)
    X_cat_df = pd.DataFrame(X_cat_encoded, columns=encoder.get_feature_names_out(cat_cols), index=df.index)

    X = pd.concat([X_num, X_cat_df], axis=1)
    y = df["EXITO"]

    # Split original
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    # 🧪 Aplicamos SMOTE
    sm = SMOTE(random_state=42)
    X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

    print(f"Nuevo balance SMOTE: {pd.Series(y_train_bal).value_counts().to_dict()}")

    # Hiperparámetros
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [3, 5],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)

    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='roc_auc',
        cv=5,
        verbose=0,
        n_jobs=-1
    )

    # Entrenamos
    grid_search.fit(X_train_bal, y_train_bal)
    model = grid_search.best_estimator_

    # Evaluamos
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {acc:.4f}")
    
    return model, encoder, cat_cols, num_cols


In [ ]:
def predecir_tienda_nueva(nueva_tienda, model, encoder, cat_cols, num_cols):
    # Armo un DataFrame con los datos que el usuario me da (la nueva tienda)
    nueva_df = pd.DataFrame([nueva_tienda])
    
    # Codifico las columnas categóricas con el encoder que ya entrenamos (para que el modelo entienda)
    cat_encoded = encoder.transform(nueva_df[cat_cols])
    
    # Transformo esa codificación en un DataFrame para poder combinarlo fácil con los numéricos
    cat_encoded_df = pd.DataFrame(cat_encoded, columns=encoder.get_feature_names_out(cat_cols), index=nueva_df.index)

    # Junto las columnas numéricas originales con las columnas categóricas codificadas
    X_new = pd.concat([nueva_df[num_cols], cat_encoded_df], axis=1)
    
    # Verifico que todas las columnas que espera el modelo estén en X_new, si falta alguna la lleno con ceros
    #    Esto pasa porque el encoder puede crear más columnas que no estaban en la nueva tienda (categorías que no tiene)
    for col in model.get_booster().feature_names:
        if col not in X_new.columns:
            X_new[col] = 0
            
    # Reordeno las columnas para que coincidan exactamente con el orden que espera el modelo (muy importante)
    X_new = X_new[model.get_booster().feature_names]
    
    # Hago la predicción: pred es la clase (éxito o no éxito)
    pred = model.predict(X_new)[0]
    
    # Obtengo la probabilidad de que sea éxito (la clase 1), para tener un “porcentaje” de confianza
    prob = model.predict_proba(X_new)[0][1]
    
    # Devuelvo la predicción y la probabilidad para que el que llamó la función decida qué hacer
    return pred, prob


In [ ]:
# Primero entreno el modelo y tomo el codificador y las columnas que necesito
modelo, codificador, cat_cols, num_cols = entrenar_modelo(df)

# Copio el DataFrame COMPLETO Y ORIGINAL limpio para usarlo sin arruinar el original
df_features = df_clean.copy()

# Creo un modelo de vecinos más cercanos, para encontrar la tienda más cercana. El vecino mas cercano (1)
knn = NearestNeighbors(n_neighbors=1)  
# Entreno el knn usando las coordenadas latitud y longitud de las tiendas
knn.fit(df_features[["LATITUD_NUM", "LONGITUD_NUM"]])

In [ ]:
# Función que recibe una nueva latitud y longitud para predecir si abrir una tienda en esa ubicación sería exitoso
def predecir_con_coordenadas(lat, lon):

    # Busco la tienda existente más cercana a las coordenadas nuevas usando KNN
    _, idx = knn.kneighbors([[lat, lon]])

    # Selecciono la fila de esa tienda más cercana
    tienda_cercana = df_features.iloc[idx[0][0]]

    # Armo un nuevo diccionario con las características de la tienda cercana, pero reemplazo su latitud y longitud por las que me dio el usuario. Esto me permite aprovechar un contexto ya existente (como entorno, nivel socioeconómico, etc.) para simular cómo le iría a una tienda con esas coordenadas.
    nueva_tienda = {
        "PLAZA_CVE": tienda_cercana["PLAZA_CVE"],
        "NIVELSOCIOECONOMICO_DES": tienda_cercana["NIVELSOCIOECONOMICO_DES"],
        "ENTORNO_DES": tienda_cercana["ENTORNO_DES"],
        "SEGMENTO_MAESTRO_DESC": tienda_cercana["SEGMENTO_MAESTRO_DESC"],
        "LID_UBICACION_TIENDA": tienda_cercana["LID_UBICACION_TIENDA"],
        "MTS2VENTAS_NUM": tienda_cercana["MTS2VENTAS_NUM"],
        "PUERTASREFRIG_NUM": tienda_cercana["PUERTASREFRIG_NUM"],
        "CAJONESESTACIONAMIENTO_NUM": tienda_cercana["CAJONESESTACIONAMIENTO_NUM"],

        # Aquí actualizo la latitud y longitud a las coordenadas que el usuario quiere evaluar
        "LATITUD_NUM": lat,   
        "LONGITUD_NUM": lon    
    }

    # Llamo a mi función de predicción para saber si esta tienda “simulada” tendría éxito en esas coordenadas
    return predecir_tienda_nueva(nueva_tienda, modelo, codificador, cat_cols, num_cols)


# Optimizacion

In [ ]:
def ventas_estimadas_con_ubicacion(x, modelo, base_tienda, encoder, cat_cols, num_cols):

    # Hago una copia de la tienda base para no modificar el original
    tienda = base_tienda.copy()

    # Le asigno a la tienda los valores nuevos que quiero probar (los que el optimizador me dice)
    tienda["MTS2VENTAS_NUM"] = x[0]
    tienda["PUERTASREFRIG_NUM"] = x[1]
    tienda["CAJONESESTACIONAMIENTO_NUM"] = x[2]

    # Uso mi función que ya predice para tiendas nuevas, así que la aprovecho
    _, prob = predecir_tienda_nueva(tienda, modelo, encoder, cat_cols, num_cols)

    # Como la función de optimización (minimize) busca minimizar, yo regreso el negativo de la probabilidad para que internamente se maximice la probabilidad de éxito
    return -prob


In [ ]:
# Defino los entornos válidos que acepto para la predicción
ENTORNOS_VALIDOS = {"Base", "Hogar", "Peatonal", "Receso"}

def predecir_y_optimizar_con_recomendacion(lat, lon, entorno, radio_km=1):
    # Primero chequeo que el entorno esté dentro de los permitidos, si no, chao
    if entorno not in ENTORNOS_VALIDOS:
        print(f"Error: entorno '{entorno}' no válido. Elige entre {ENTORNOS_VALIDOS}")
        return None

    # Verifico que la ubicación esté dentro de México, no le hacemos predicción si está fuera
    if not esta_en_mexico(lat, lon):
        print("Ubicación fuera de México, no se puede hacer predicción.")
        return None
    
    # Busco la tienda más cercana a las coordenadas que me dieron
    _, idx = knn.kneighbors([[lat, lon]])
    tienda_cercana = df_features.iloc[idx[0][0]]

    # Armo la base de datos con las características de esa tienda cercana,
    # pero uso el entorno que me pasaron (puede cambiar) y las coordenadas nuevas
    base_tienda = {
        "PLAZA_CVE": tienda_cercana["PLAZA_CVE"],
        "NIVELSOCIOECONOMICO_DES": tienda_cercana["NIVELSOCIOECONOMICO_DES"],
        "ENTORNO_DES": entorno,
        "SEGMENTO_MAESTRO_DESC": tienda_cercana["SEGMENTO_MAESTRO_DESC"],
        "LID_UBICACION_TIENDA": tienda_cercana["LID_UBICACION_TIENDA"],
        "MTS2VENTAS_NUM": tienda_cercana["MTS2VENTAS_NUM"],
        "PUERTASREFRIG_NUM": tienda_cercana["PUERTASREFRIG_NUM"],
        "CAJONESESTACIONAMIENTO_NUM": tienda_cercana["CAJONESESTACIONAMIENTO_NUM"],
        "LATITUD_NUM": lat,
        "LONGITUD_NUM": lon
    }

    # Valores iniciales para optimizar (metros cuadrados, puertas y cajones)
    x0 = [base_tienda["MTS2VENTAS_NUM"], base_tienda["PUERTASREFRIG_NUM"], base_tienda["CAJONESESTACIONAMIENTO_NUM"]]
    bounds = [(50, 200), (1, 10), (0, 10)]

    # Le pido al optimizador que me busque esos valores para maximizar la probabilidad de éxito
    resultado = minimize(ventas_estimadas_con_ubicacion, x0, args=(modelo, base_tienda, codificador, cat_cols, num_cols),
                         bounds=bounds, method='L-BFGS-B')

    # Extraigo los valores óptimos y la probabilidad máxima (ojo que minimize devuelve negativo para minimizar)
    mts2_opt, puertas_opt, cajones_opt = resultado.x
    prob_opt = -resultado.fun

    # Predicción con los parámetros base, para comparar
    pred_usuario, prob_init = predecir_tienda_nueva(base_tienda, modelo, codificador, cat_cols, num_cols)

    print("\nRESULTADO INICIAL CON PARÁMETROS BASE:")
    print(f"Predicción: {'Éxito (1)' if pred_usuario == 1 else 'No Éxito (0)'}")
    print(f"Probabilidad de éxito: {prob_init:.2%}")
    
    # Veo qué tan lejos está la venta real de la meta para esta tienda base
    venta_real = tienda_cercana["VENTA_TOTAL"]
    meta_real = tienda_cercana["Meta_venta"]
    diferencia = ((venta_real - meta_real) / meta_real) * 100

    if pred_usuario == 1:
        print(f"Las ventas mejorarían en aproximadamente {diferencia:.2f}%")
    else:
        print(f"Las ventas caerían por debajo de la meta en aproximadamente {abs(diferencia):.2f}%")

    print("\nRESULTADO OPTIMIZADO:")
    print(f"m2 óptimo: {mts2_opt:.2f}")
    print(f"Puertas de Refri óptimo: {puertas_opt:.2f}")
    print(f"Cajones de estacionamiento óptimo: {cajones_opt:.2f}")
    print(f"Probabilidad estimada de éxito: {prob_opt:.2%}")

    # Ahora busco tiendas vecinas en el radio especificado para recomendar mejor ubicación
    df_features["DISTANCIA"] = haversine(lat, lon, df_features["LATITUD_NUM"], df_features["LONGITUD_NUM"])
    vecinas = df_features[df_features["DISTANCIA"] <= radio_km]

    mejor_prob = prob_init
    mejor_ubicacion = (lat, lon, prob_init)

    # Reviso cada tienda vecina para ver si hay una mejor opción con +5% probabilidad
    for _, row in vecinas.iterrows():
        tienda = {
            "PLAZA_CVE": row["PLAZA_CVE"],
            "NIVELSOCIOECONOMICO_DES": row["NIVELSOCIOECONOMICO_DES"],
            "ENTORNO_DES": entorno,
            "SEGMENTO_MAESTRO_DESC": row["SEGMENTO_MAESTRO_DESC"],
            "LID_UBICACION_TIENDA": row["LID_UBICACION_TIENDA"],
            "MTS2VENTAS_NUM": row["MTS2VENTAS_NUM"],
            "PUERTASREFRIG_NUM": row["PUERTASREFRIG_NUM"],
            "CAJONESESTACIONAMIENTO_NUM": row["CAJONESESTACIONAMIENTO_NUM"],
            "LATITUD_NUM": row["LATITUD_NUM"],
            "LONGITUD_NUM": row["LONGITUD_NUM"]
        }
        _, prob = predecir_tienda_nueva(tienda, modelo, codificador, cat_cols, num_cols)
        if prob > mejor_prob + 0.05:  # Solo cambio si la mejora es más del 5%
            mejor_prob = prob
            mejor_ubicacion = (row["LATITUD_NUM"], row["LONGITUD_NUM"], prob)

    # Finalmente imprimo la recomendación si encontré mejor opción
    if mejor_ubicacion[0] != lat or mejor_ubicacion[1] != lon:
        lat2, lon2, prob2 = mejor_ubicacion
        print("\nRECOMENDACIÓN:")
        print(f"Ubicación sugerida: ({lat2:.5f}, {lon2:.5f})")
        print(f"Probabilidad de éxito ahí: {prob2:.2%} (+{(prob2 - prob_init):.2%})")
    else:
        print("\nEsta ubicación es adecuada. No se encontró una mejora significativa en el radio de búsqueda.")


# P R U E B A

In [ ]:
predecir_y_optimizar_con_recomendacion(25.653587733944487, -100.39367061990951, "Base")